In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import theano.tensor as T

In [3]:
from fuel.datasets.cifar10 import CIFAR10
from fuel.transformers import ScaleAndShift, Cast, Flatten
from fuel.streams import DataStream
from fuel.schemes import SequentialScheme, ShuffledScheme

CIFAR10.default_transformers = (
    (ScaleAndShift, [2.0 / 255.0, -1], {'which_sources': 'features'}),
    (Cast, [np.float32], {'which_sources': 'features'}) ,
    (Flatten, [], {'which_sources': 'features'}) )

cifar_train = CIFAR10(("train",), subset=slice(None,40000))
cifar_train_stream = DataStream.default_stream(
    cifar_train, iteration_scheme=ShuffledScheme(cifar_train.num_examples, 50))
                                               
cifar_validation = CIFAR10(("train",), subset=slice(40000, None))
cifar_validation_stream = DataStream.default_stream(
    cifar_validation, iteration_scheme=SequentialScheme(cifar_validation.num_examples, 100))

cifar_test = CIFAR10(("test",))
cifar_test_stream = DataStream.default_stream(
    cifar_test, iteration_scheme=SequentialScheme(cifar_test.num_examples, 250))

In [4]:
from modules.Network import Network
from modules.layers.SoftMaxLayer import SoftMaxLayer
from modules.layers.AffineLayer import AffineLayer
from modules.layers.ConvolutionalPoolLayer import ConvolutionalPoolLayer

In [5]:
ReLU = lambda x: T.maximum(0.0, x)

net = Network([ConvolutionalPoolLayer(image_shape=(3,32,32), filter_shape=(20,3,5,5), pool_shape=(2,2)),
               ConvolutionalPoolLayer(image_shape=(20,14,14), filter_shape=(40,20,5,5), pool_shape=(2,2)),
               AffineLayer(40*5*5, 500, activation=ReLU),
               SoftMaxLayer(500, 10)])
net.train(cifar_train_stream, cifar_validation_stream, 2, 4e-3, 0.9, 1e-3)

At batch #100, batch cost: 2.218246
At batch #200, batch cost: 2.008744
At batch #300, batch cost: 1.890899
At batch #400, batch cost: 1.721766
At batch #500, batch cost: 1.670758
At batch #600, batch cost: 1.747774
At batch #700, batch cost: 1.606788
At batch #800, batch cost: 1.465501
After epoch 1: validation error: 55.230000%
At batch #900, batch cost: 1.432447
At batch #1000, batch cost: 1.260487
At batch #1100, batch cost: 1.383768
At batch #1200, batch cost: 1.562953
At batch #1300, batch cost: 1.385300
At batch #1400, batch cost: 1.210097
At batch #1500, batch cost: 1.578728
At batch #1600, batch cost: 1.257366
After epoch 2: validation error: 49.150000%


In [7]:
net.compute_error_rate(cifar_test_stream)

0.4854